In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import graphviz 
import pydotplus
from sklearn.ensemble import AdaBoostRegressor
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import NuSVR, LinearSVR
from sklearn.svm import SVC 
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.cluster import KMeans

In [2]:

data = pd.read_csv('DC_Properties.csv')

data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (18,19,20,21,22,23,24,33,34,35,37,43,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,BATHRM,HF_BATHRM,HEAT,AC,NUM_UNITS,ROOMS,BEDRM,AYB,YR_RMDL,...,LONGITUDE,ASSESSMENT_NBHD,ASSESSMENT_SUBNBHD,CENSUS_TRACT,CENSUS_BLOCK,WARD,SQUARE,X,Y,QUADRANT
0,0,4,0,Warm Cool,Y,2.0,8,4,1910.0,1988.0,...,-77.040832,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
1,1,3,1,Warm Cool,Y,2.0,11,5,1898.0,2007.0,...,-77.040764,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
2,2,3,1,Hot Water Rad,Y,2.0,9,5,1910.0,2009.0,...,-77.040678,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
3,3,3,1,Hot Water Rad,Y,2.0,8,5,1900.0,2003.0,...,-77.040629,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW
4,4,2,1,Warm Cool,Y,1.0,11,3,1913.0,2012.0,...,-77.039361,Old City 2,040 D Old City 2,4201.0,004201 2006,Ward 2,152,-77.040429,38.914881,NW


In [3]:
data["SOURCE"] = "Residential"
data["QUALIFIED"] = 'Q'
data_limited = data[(data['STORIES'] < 6) & (data['PRICE'] >99000) & (data['PRICE'] <5000000) & (data['SALEDATE'] > '2013-07-01')]

data_viz = data_limited.drop(['NUM_UNITS','AYB','YR_RMDL','SALE_NUM','BLDG_NUM','GIS_LAST_MOD_DTTM','CMPLX_NUM','LIVING_GBA','NATIONALGRID'], axis=1)

data_ml = data_limited.drop(['NUM_UNITS','AYB','YR_RMDL','SALE_NUM','BLDG_NUM','GIS_LAST_MOD_DTTM','CMPLX_NUM','LIVING_GBA','CITY',
                    'STATE','NATIONALGRID','FULLADDRESS','ASSESSMENT_NBHD','ASSESSMENT_SUBNBHD','CENSUS_TRACT','CENSUS_BLOCK',
                    'WARD','SQUARE','X','Y','SOURCE','QUALIFIED','Unnamed: 0','LATITUDE','LONGITUDE', 'STYLE'], axis=1)

In [4]:

encoding = {'GRADE':{'Very Good':4, 'Above Average':6, 'Good Quality':3, 'Excellent':8, 'Average':5, 'Superior':7,
                     'Fair Quality':2, 'Exceptional-D':12, 'Exceptional-C':11, 'Low Quality':1, 'Exceptional-A':9,
                     'Exceptional-B':10, 'No Data':0},
            'CNDTN':{'Good':4, 'Very Good':5, 'Average':3, 'Fair':2, 'Excellent':6, 'Poor':1, 'Default':0}}

data_ml.replace(encoding, inplace=True)

data_ml

,BATHRM,HF_BATHRM,HEAT,AC,ROOMS,BEDRM,EYB,STORIES,SALEDATE,PRICE,...,CNDTN,EXTWALL,ROOF,INTWALL,KITCHENS,FIREPLACES,USECODE,LANDAREA,ZIPCODE,QUADRANT
2,3,1,Hot Water Rad,Y,9,5,1984,3.0,2016-06-21 00:00:00,2100000.0,...,5,Common Brick,Built Up,Hardwood,2.0,4,24,1680,20009.0,NW
8,3,1,Warm Cool,Y,7,3,1967,2.0,2018-05-03 00:00:00,1430000.0,...,5,Common Brick,Built Up,Hardwood,2.0,1,24,1424,20009.0,NW
29,3,1,Forced Air,Y,11,3,1984,3.0,2013-12-11 00:00:00,1210000.0,...,5,Common Brick,Built Up,Hardwood,2.0,3,11,1853,20009.0,NW
41,3,1,Warm Cool,Y,9,4,1967,3.0,2016-09-14 00:00:00,1550000.0,...,5,Common Brick,Metal- Sms,Hardwood,2.0,2,24,1853,20009.0,NW
42,3,1,Forced Air,Y,9,4,1967,3.0,2015-09-16 00:00:00,1599900.0,...,4,Common Brick,Built Up,Hardwood,2.0,2,24,1853,20009.0,NW
43,3,1,Forced Air,Y,13,4,1967,3.0,2018-03-30 00:00:00,1700000.0,...,6,Common Brick,Metal- Sms,Hardwood,2.0,3,24,1853,20009.0,NW
44,3,2,Forced Air,Y,10,4,1967,3.0,2017-05-01 00:00:00,1700000.0,...,5,Common Brick,Built Up,Hardwood,2.0,2,24,1853,20009.0,NW
62,4,2,Elec Base Brd,Y,11,5,1978,4.0,2014-03-05 00:00:00,1599000.0,...,4,Common Brick,Built Up,Hardwood/Carp,3.0,0,24,950,20009.0,NW
72,4,2,Warm Cool,Y,8,3,1964,2.0,2014-10-08 00:00:00,993500.0,...,5,Common Brick,Built Up,Hardwood,1.0,1,11,814,20009.0,NW
78,2,1,Warm Cool,Y,11,4,1990,2.0,2013-10-30 00:00:00,1155000.0,...,5,Common Brick,Built Up,Hardwood,2.0,2,24,1183,20009.0,NW


In [5]:

data_ml_dummy = pd.get_dummies(data_ml, columns=['HEAT','AC','STRUCT','EXTWALL','ROOF','INTWALL','USECODE','ZIPCODE','QUADRANT'])
data_ml_dummy

,BATHRM,HF_BATHRM,ROOMS,BEDRM,EYB,STORIES,SALEDATE,PRICE,GBA,GRADE,...,ZIPCODE_20019.0,ZIPCODE_20020.0,ZIPCODE_20024.0,ZIPCODE_20032.0,ZIPCODE_20036.0,ZIPCODE_20037.0,QUADRANT_NE,QUADRANT_NW,QUADRANT_SE,QUADRANT_SW
2,3,1,9,5,1984,3.0,2016-06-21 00:00:00,2100000.0,2522.0,4,...,0,0,0,0,0,0,0,1,0,0
8,3,1,7,3,1967,2.0,2018-05-03 00:00:00,1430000.0,1488.0,6,...,0,0,0,0,0,0,0,1,0,0
29,3,1,11,3,1984,3.0,2013-12-11 00:00:00,1210000.0,2520.0,6,...,0,0,0,0,0,0,0,1,0,0
41,3,1,9,4,1967,3.0,2016-09-14 00:00:00,1550000.0,2552.0,6,...,0,0,0,0,0,0,0,1,0,0
42,3,1,9,4,1967,3.0,2015-09-16 00:00:00,1599900.0,2520.0,6,...,0,0,0,0,0,0,0,1,0,0
43,3,1,13,4,1967,3.0,2018-03-30 00:00:00,1700000.0,2382.0,6,...,0,0,0,0,0,0,0,1,0,0
44,3,2,10,4,1967,3.0,2017-05-01 00:00:00,1700000.0,2520.0,6,...,0,0,0,0,0,0,0,1,0,0
62,4,2,11,5,1978,4.0,2014-03-05 00:00:00,1599000.0,3162.0,8,...,0,0,0,0,0,0,0,1,0,0
72,4,2,8,3,1964,2.0,2014-10-08 00:00:00,993500.0,1148.0,5,...,0,0,0,0,0,0,0,1,0,0
78,2,1,11,4,1990,2.0,2013-10-30 00:00:00,1155000.0,1380.0,5,...,0,0,0,0,0,0,0,1,0,0


In [6]:
data_ml_dummy['SALEDATE']=pd.to_datetime(data_ml_dummy['SALEDATE']).dt.year
data_ml_dummy = data_ml_dummy.dropna()

In [7]:

X = data_ml_dummy.drop('PRICE', axis=1).values
y = data_ml_dummy["PRICE"].values.reshape(-1, 1)

# print(X.shape, y.shape)

In [8]:
# data_ml_dummy['LOGPRICE']=np.log(data_ml_dummy['PRICE'])

In [9]:
# X=data_ml_dummy.drop(['PRICE','LOGPRICE'], axis=1)
# y=data_ml_dummy['LOGPRICE']

In [10]:
# X = df_clean.drop('PRICE', axis=1).values
# y = df_clean["PRICE"].values.reshape(-1, 1)
# # X = voice.drop("label", axis=1)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 42)

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [12]:

def modelMaker(df, target, models, scale = True, dummy = True, rs = 42):
    global X_train_pca, X_test_pca, y_train, y_test
    try:
        assert len(models) > 0

    except:
        models = [models]

    for model in models:
        print("Fitting "+str(model).split("(")[0])

        print("R^2: "+ str((X_test, y_test)))
        
        for n in range(2, 60, 2):
            pca = PCA(n_components=n)

            # note, y is optional here
            pca.fit(X_train_scaled)
            X_train_pca=pca.transform(X_train_scaled)
            X_test_pca =pca.transform(X_test_scaled)
            model.fit(X_train_pca, y_train)

            print()
            print("n: "+ str(n))

            print("R^2: "+ str(model.score(X_test_pca, y_test)))
            
modelMaker(data_ml_dummy, "PRICE", [LinearRegression(), Lasso(), Ridge(), ElasticNet(), NuSVR(), LinearSVR()])

In [13]:
# NuSVR seems to render highest at around n=26

In [14]:
#PCA by itself after find best components
pca = PCA(n_components=26)

# note, y is optional here
pca.fit(X_train_scaled)        
X_train_pca=pca.transform(X_train_scaled)   
X_test_pca =pca.transform(X_test_scaled)   
pca.fit(X_train_pca, y_train)

PCA(copy=True, iterated_power='auto', n_components=26, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [15]:
# NUSVR = NuSVR()
# model = NuSVR(kernel='linear')
# model

# param_grid = {'C': [1, 5, 10, 50],
#               'gamma': [0.0001, 0.0005, 0.001, 0.005],
#              'kernel': ['rbf', 'poly', 'sigmoid', 'precomputed' ]}
# grid = GridSearchCV(model, param_grid, verbose=3)

In [16]:
# clf = NuSVR(gamma='scale', C=1.0, nu=0.1)
# clf.fit(X_train, y_train)

In [17]:
# clf.predict(X_test)

In [18]:
# model = SVR()
# model = SVR(kernel='linear')
# model

# param_grid = {'C': [50],
#               'gamma': [0.0005],
#              'kernel': ['rbf']}
# grid = GridSearchCV(model, param_grid, verbose=3, cv=5)

In [19]:
# model_SVR = SVR(kernel=('rbf'), C=50, gamma=0.0005)

In [20]:
# model_SVR.fit(X_train_scaled, y_train)

In [21]:
# model_SVR.score(X_test_scaled, y_test)

In [22]:
# y_Pred = model_SVR.predict(X_test_scaled)


In [23]:
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_Pred))  
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_Pred))  
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_Pred))) 

In [24]:
# grid.fit(X_train_scaled, y_train)

In [25]:
# grid.score(X_test, y_test)

In [26]:
# grid.predict(X_test)

In [27]:
# print(grid.best_params_)

In [28]:
# print(grid.best_score_)

In [29]:
# predictions = grid.predict(X_test)

# Decision Tree Regessor

In [30]:
# # Testing Score of .73
regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X_train_scaled, y_train)


In [31]:
#     print("Training score:{0}".format(regressor.score(X_train,y_train)))
print("Testing score:{0}".format(regressor.score(X_test_scaled,y_test)))
#     ##graph

In [32]:
y_Pred_test = regressor.predict(X_test_scaled)


In [33]:
# r2_score(y_true, y_pred_test)

In [34]:
nndf=pd.DataFrame(y_test)
nndf['PRED']=y_Pred_test
nndf=nndf.reset_index(drop=True)

In [35]:
0 columm is test data
nndf

In [37]:
plt.plot(nndf.index,nndf[0])
plt.plot(nndf.index,nndf['PRED'], color='red', alpha=0.7)
plt.xlabel('Index')
plt.ylabel('Price-test')
plt.title('Decision Tree Regression Actual (Blue) vs Predicted (Red)')
plt.show()

In [36]:
nndf.rename({0:'ACT'}, axis=1, inplace=True)

In [38]:
# plt.savefig('Decission_tree.png')

In [39]:
# y_pred=regressor.predict(X_test)

In [40]:
# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
# print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

In [41]:
# print(confusion_matrix(y_test,y_Pred))  
# print(classification_report(y_test,y_Pred))  
# print(accuracy_score(y_test, y_Pred)) 

# KNN Regression

In [107]:
# Testing Score of .76
knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train_scaled, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=10, p=2,
          weights='uniform')

In [108]:
y_pred = knn.predict(X_test_scaled)


In [109]:
print("Testing score:{0}".format(knn.score(X_test_scaled,y_test)))

Testing score:0.7606576372481202


In [44]:
# r2_score(y_test, y_pred)

In [45]:
nndf=pd.DataFrame(y_test)
nndf['PRED']=y_pred
nndf=nndf.reset_index(drop=True)

In [47]:
plt.plot(nndf.index,nndf[0])
plt.plot(nndf.index,nndf['PRED'], color='red' ,alpha = 0.7)
plt.xlabel('Index')
plt.ylabel('Price-test')
plt.title('KNN Regression Actual (Blue) vs Predicted (Red)')
plt.show()

In [46]:
nndf.rename({0:'ACT'}, axis=1, inplace=True)

In [48]:
# plt.savefig('KNN_regression.jpg')

# Linear Regression 

In [49]:
# lm = LinearRegression()
# lm.fit(X_train_scaled, y_train)

In [50]:
# y_pred = lm.predict(X_test_scaled)

In [51]:
# r2_score(y_test, y_pred)

In [52]:
# nndf=pd.DataFrame(y_test)
# nndf['PRED']=y_pred
# nndf=nndf.reset_index(drop=True)

In [53]:
# plt.plot(nndf.index,nndf[0])
# plt.plot(nndf.index,nndf['PRED'], color='red')
# plt.xlabel('Index')
# plt.ylabel('Price-test')
# plt.title('KNN Regression Actual (Blue) vs Predicted (Red)')
# plt.show()

# AdaBoost Regression

In [54]:
regr = AdaBoostRegressor(random_state=42, n_estimators=50)
regr.fit(X_train_scaled, y_train)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=42)

In [55]:
regr.fit(X_train_scaled, y_train) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=42)

In [105]:
print("Testing score:{0}".format(regr.score(X_test_scaled,y_test)))

Testing score:-0.469447007188488


In [56]:
 y_pred = regr.predict(X_test_scaled)

In [57]:
nndf=pd.DataFrame(y_test)
nndf['PRED']=y_pred
nndf=nndf.reset_index(drop=True)

In [58]:
plt.plot(nndf.index,nndf[0])
plt.plot(nndf.index,nndf['PRED'], color='red')
plt.xlabel('Index')
plt.ylabel('Price-test')
plt.title('AdaBooster Regression Actual (Blue) vs Predicted (Red)')
plt.show()

In [59]:
# y_pred = lm.predict(X_test_scaled)

In [60]:
# r2_score(y_test, y_pred)

# Nueral Network

In [101]:
model = Sequential()
model.add(Dense(units=116, activation='relu', input_dim=116))
model.add(Dense(units=240, activation='softmax'))

In [102]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [104]:
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=200
)

Epoch 1/100


InvalidArgumentError: Received a label value of 1860000 which is outside the valid range of [0, 240).  Label values: 625000 331140 595000 610000 1350000 359500 165000 935000 200000 1075000 1150000 270250 516500 239000 274900 130000 974000 1860000 182500 1425889 1000000 632545 985000 960000 476050 725000 882000 350000 279000 640000 958000 759000
	 [[{{node loss_5/dense_13_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]

# K-Means
## Used for unsupervised Learning not good model

In [79]:
# kmeans = KMeans(n_clusters=8)

In [80]:
# kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [81]:
# predicted_clusters = kmeans.predict(X)

In [82]:
# plt.scatter(nndf.index,nndf[0], nndf.index,nndf['PRED'], c=predicted_clusters, s=50, cmap='viridis')

TypeError: scatter() got multiple values for argument 'c'

In [72]:
# y_pred = kmeans.predict(X_test_scaled)

In [73]:
# nndf=pd.DataFrame(y_test)
# nndf['PRED']=y_pred
# nndf=nndf.reset_index(drop=True)

In [ ]:
# plt.plot(nndf.index,nndf[0])
# plt.plot(nndf.index,nndf['PRED'], color='red')
# plt.xlabel('Index')
# plt.ylabel('Price-test')
# plt.title('AdaBooster Regression Actual (Blue) vs Predicted (Red)')
# plt.show()